<a href="https://colab.research.google.com/github/luimui/DataScience/blob/main/part1_tasks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# An immune clock of human pregnancy

## Load the data

Load the data from `multiomics_data.pickle` using `pickle`. You will get a [pandas](https://pandas.pydata.org/docs/user_guide/10min.html) DataFrame containing preprocessed data from the paper (the original data from their paper is a bit messy). The data contains several meta attributes as well as the different modalities.

Meta attributes include:

* `Gates ID`, `MRN`, `Study Subject ID Number`: different IDs of a patient
* `Sex`: sex of the baby
* `timepoint`: 1-3 correspond to the three trimesters, 4 corresponds to postpartum
* `gestational_age`: time of sampling

There are many different modalities. We only want to focus on the `immune_system`:
    
* `cellfree_rna`
* `metabolomics`
* `microbiome`
* `plasma_luminex`
* `serum_luminex`
* `immune_system`
* `plasma_somalogic`

If you are interested in the rest, take a look at [this paper](https://academic.oup.com/bioinformatics/article/35/1/95/5047759).

In [4]:
# code for loading the data

import numpy as np
import pickle

from google.colab import drive
drive.mount('/content/drive')

with open("/content/drive/MyDrive/DataScienceWS202324/Project/part1_multiomics_data.pickle", "rb") as file:
    data_multiomics = pickle.load(file)

data_multiomics.head(5)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Training/Validation Gates ID       MRN Study Subject ID Number     Sex  \
                                                                           
0                   T  PTLG002  16661779                   10565    Male   
1                   T  PTLG002  16661779                   10565    Male   
2                   T  PTLG002  16661779                   10565    Male   
3                   T  PTLG002  16661779                   10565    Male   
4                   T  PTLG004  23587868                   10603  Female   

  sex_bin timepoint gestational_age cellfree_rna                ...  \
                                       0_C2orf76      1_ACTL10  ...   
0       1         1              11     0.312437 -1.892930e-16  ...   
1       1         2              18     0.312437 -1.892930e-16  ...   
2       1         3              32     0.312437 -1.892930e-16  ...   
3       1         4              45     0.312437 -1.892930e-16  ...   
4       0         1              11     5.204209  1.734736e+00  ...   

  plasma_somalogic                                                             \
       1290_UBE2G2 1291_TAGLN2 1292_ATP5O 1293_POMC 1294_CRYZL1 1295_SERPINF1   
0           4804.4      2233.0     3610.9     715.8       151.4       37885.8   
1           4086.0      2160.5     2260.4     825.2       161.0       41821.5   
2           4328.0      1818.4     2445.2    1241.8       194.6       45526.1   
3           3442.4      2661.4     3879.2     703.6       153.7       36862.5   
4           4261.9      1804.6     1470.6     526.8       163.0       38938.3   

                                              
  1296_CTSF 1297_FTCD 1298_USP25 1299_PLXNB2  
0    1479.1    3261.8      561.3      3227.0  
1    1465.1    1839.8      597.8      3366.0  
2    1428.3    3057.2      625.7      8703.7  
3    1063.6    7339.7      593.2      2918.9  
4    1170.1    1036.8      552.8      3457.1  

[5 rows x 61274 columns]

In [24]:

# look at the immune system
data_multiomics["immune_system"].head(5)
data = data_multiomics["immune_system"]

data.isnull().values.any()

data.dtypes
data.head(5)
data.shape

(68, 534)

## Task 1: Predict Gestational Age

Use what you learned in the lecture and the tutorials to:

1. **Predict `gestational_age`** using the `immune_system` modality using at least two models (e.g., LinearRegression, LASSO, Ridge, ElasticNet, ...)

2. **Evaluate** your models using a measure that you think fits best and report the result. If it is a different measure than in the paper, please briefly explain why.

In [82]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn import metrics
from matplotlib import pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import cross_val_score
from sklearn.feature_selection import RFE
from sklearn.model_selection import GridSearchCV


X = data
#X = X.sample(n=5,axis='columns')
y = data_multiomics["gestational_age"]



X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)
X_train

model = LinearRegression().fit(X_train, y_train)


hyper_params = [{'n_features_to_select': list(range(1, X.shape[1]))}]

rfe = RFE(model)
model_cv = GridSearchCV(estimator = rfe,
                        param_grid = hyper_params,
                        scoring= 'r2',
                        cv = 2,
                        verbose = 1,
                        return_train_score=True)


model_cv.fit(X_train, y_train)

y_pred = model_cv.predict(X_test)

# model evaluation for testing set

mae = metrics.mean_absolute_error(y_test, y_pred)
mse = metrics.mean_squared_error(y_test, y_pred)
r2 = metrics.r2_score(y_test, y_pred)

print("The model performance for testing set")
print("--------------------------------------")
print('MAE is {}'.format(mae))
print('MSE is {}'.format(mse))
print('R2 score is {}'.format(r2))

print("score: {}".format(model.score(X_test, y_test)))
#print("coeff: {}".format(model.coef_))
print("intercept: {}".format(model.intercept_))


plt.scatter(y_test,y_pred,c="blue",marker="*")


scores = cross_val_score(model, X_train, y_train, scoring='r2', cv=5)
print(scores)

(54, 534) (14, 534) (54,) (14,)
Fitting 2 folds for each of 533 candidates, totalling 1066 fits


KeyboardInterrupt: ignored

In [76]:
X.shape


(68, 5)

## Task 2: Read the paper and explore the data

Think about and answer the following questions:

1. How many samples are there in the dataset?
2. How many pregnant women are there in the dataset?
3. What are the timepoints of the samples?
4. How many features are there in the dataset? Describe the feature groups.

## Bonus: Sex of the baby

Predict the sex of the baby. Try using a neural network in Tensorflow or PyTorch if you feel adventurous (here is a [tutorial](https://pytorch.org/tutorials/beginner/blitz/cifar10_tutorial.html?highlight=data%20loader)).